In [2]:
import numpy as np
import numpy.random
import matplotlib.pyplot as plt

class Model:
    def __init__(self):
        self.weight = np.zeros(100)
        
    def __call__(self, x: np.ndarray):
        x = (self.weight * x).sum(axis=1, keepdims=True)
        x = 1 / (1 + np.exp(-x)) 
        return x
    
    def __str__(self):
        return f'{self.weight}'

def BCELoss(pred: np.ndarray, label: np.ndarray):
    return -(label * np.log(pred) + (1 - label) * np.log(1 - pred)).mean(dtype=np.float64)

In [4]:
def preprocess_data(data):
    r'''
    compute input data of model base on g(x, y; \theta)
    '''
    transformed_data = data
    for i in range(10):
        for j in range(10):
            transformed_data = np.concatenate((transformed_data, data[:, 0:1] ** i * data[:, 1:2] ** j), axis=1)
    return transformed_data[:, 2:]


data = np.genfromtxt("data-nonlinear.txt", delimiter=',')
y_train = data[:, 2:3]
transformed_data = preprocess_data(data[:, 0:2])
transformed_data.shape

(118, 100)

In [6]:
from tqdm import tqdm

epoch = 20000
max_acc = 0

losses = []
accuracys = []
weights = []

losse = []
accuracy = []

learning_rate = 5
weight_decay = [1e-6, 1e-3, 1e-1]
model = Model()

for i in range(3):
    loss = []
    accuracy = []
    for step in tqdm(range(epoch)):
        # BCELoss
        criterion: np.ndarray = BCELoss(model(transformed_data), y_train) + weight_decay[i] * (model.weight ** 2).mean() / 2
        preds = model(transformed_data) > 0.5
        acc = (preds == y_train).sum() / y_train.shape[0]
        max_acc = max(acc.item(), max_acc)

        # Save losses and parameters
        loss += [criterion.tolist()]
        accuracy += [acc.item()]

        # Compute Error
        error = model(transformed_data) - y_train

        # Gradient Descent Algorithm
        temp_weight = (1 - learning_rate * weight_decay[i]) * model.weight - learning_rate * (error * transformed_data).mean(axis=0)

        # Update parameter
        model.weight = temp_weight
    losses += [loss]
    accuracys += [accuracy]
    weights += [model.weight]

  1%|▏         | 29433/2000000 [00:05<05:35, 5872.51it/s]


KeyboardInterrupt: 